In [3]:
import sys
sys.path.append('..')

from crisismmd_dataset import CrisisMMDataset
import os.path as osp
from torch.utils.data import DataLoader
from paths import modelroot
from tqdm import tqdm

class Args:
    pass


opt = Args()
opt.gpu = 0
opt.batch_size = 1
opt.max_dataset_size = 2000
# opt.max_dataset_size = 100
opt.crop_size = 224
opt.load_size = 224
opt.debug = False
opt.device = 'cuda'
opt.task = 'task1' 
# opt.task = 'task2_merged'
opt.consistent_only = True
opt.num_classes = 2 if opt.task == 'task1' else 6

dataset = CrisisMMDataset()
dataset.initialize(opt, phase='dev', cat='all', task=opt.task, no_transform=False, use_cate=False, consistent_only=opt.consistent_only)
data_loader = DataLoader(
    dataset, batch_size=1, shuffle=False,
)
len(data_loader)
for x in data_loader:
    print(x)
    break

[*] 1573 samples loaded.
{'path_image': ['W:/datasets/CrisisMMD_v2.0/data_image/hurricane_harvey/5_9_2017/905064623199719425_0.jpg'], 'text': ["We've lost track of how many houses/families @NEHBC teams have helped. So proud of our church. #Harvey https://t.co/wBkKvUt9vw"], 'text_tokens': {'input_ids': tensor([[  101,  2057,  1005,  2310,  2439,  2650,  1997,  2129,  2116,  3506,
          1013,  2945,  1030, 11265,  2232,  9818,  2780,  2031,  3271,  1012,
          2061,  7098,  1997,  2256,  2277,  1012,  7702,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}, 'label_str': ['informative'], 'label': tensor([1]), 'label_

In [4]:
from sklearn.svm import SVC
import numpy as np
import torch
import scipy.stats
from sklearn.metrics import accuracy_score

def p2oh(prediction):
    oh = torch.LongTensor(opt.num_classes).zero_()
    oh[prediction] = 1
    return oh.numpy()

def build_xy(extractor, data_loader, data_limit=100):
    y = []
    x = []
    for idx, data in tqdm(enumerate(data_loader), total=data_limit):
        if idx > data_limit:
            break
        
        # y.append(p2oh(data['label'].numpy()))
        y.append(data['label'].numpy())

        feat = extractor(data)
        x.append(feat)

    x = np.array(x)
    if x[0].size == 1:
        x = x.reshape(-1, 1)
    print(x)
    y = np.array(y)
    print(y)

    return x, y


def evaluate_svm(x, y):
    clf = SVC(kernel='rbf', C=0.5, gamma='scale')
    clf.fit(x, y)
    pred_y = clf.predict(x)
    true_y = y.squeeze()
    print(true_y, pred_y)
    print(accuracy_score(true_y, pred_y))
    return clf


# Reference: https://scikit-learn.org/0.18/auto_examples/svm/plot_iris.html
def plot(clf, x):
    pass 


def evaluate_corr(x, y):
    return scipy.stats.pearsonr(x, y)


In [5]:
sys.path.append('..')
from categorizer import Categorizer
categorizer = Categorizer()
x, y = build_xy(categorizer._get_feat_semantic_seg, data_loader, 100)

Loading weights for net_encoder
Loading weights for net_decoder


101it [00:12,  8.29it/s]                         

[[1.83590234e+04 6.98067522e+00 7.70717239e+00 ... 9.49469280e+00
  6.84049368e+00 9.74411774e+00]
 [6.56344849e+02 5.58814355e+03 3.21550293e+03 ... 3.28827248e+01
  2.19507484e+01 3.01751766e+01]
 [4.13773438e+04 2.94345169e+01 1.57726941e+03 ... 1.24570742e+01
  2.35578728e+00 3.31373358e+00]
 ...
 [4.14588828e+04 3.58762970e+01 2.23698196e+02 ... 2.34626732e+01
  1.86143112e+01 1.79013977e+01]
 [1.72409883e+04 8.65854785e+03 1.00935645e+03 ... 6.64463959e+01
  2.93664169e+01 3.94620743e+01]
 [3.38492812e+04 2.79579186e+00 1.24049622e+03 ... 8.34384346e+00
  4.17976713e+00 6.59412479e+00]]
[[1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]


In [6]:
evaluate_svm(x, y)

[1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 0
 0 0 0 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1
 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 1 1 0] [1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
0.7326732673267327


E:\anaconda\envs\11777\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC(C=0.5)

In [7]:
# evaluate_corr(x, y)
y_oh = np.array([p2oh(pred) for pred in y.squeeze()])
for var_idx in range(x.shape[1]):
    for y_idx in range(y_oh.shape[1]):
        corr = evaluate_corr(x[:, var_idx], y_oh[:, y_idx])[0]
        print('{} - {}: {}'.format(var_idx, y_idx, corr))

0 - 0: 0.14230062867351756
0 - 1: -0.14230062867351756
1 - 0: -0.18886741803530943
1 - 1: 0.18886741803530943
2 - 0: 0.025239072974683772
2 - 1: -0.025239072974683772
3 - 0: -0.22503941921578097
3 - 1: 0.22503941921578097
4 - 0: -0.08771454595664749
4 - 1: 0.08771454595664747
5 - 0: 0.012399449126776
5 - 1: -0.012399449126775978
6 - 0: -0.15697597108260192
6 - 1: 0.15697597108260192
7 - 0: -0.1025661268924787
7 - 1: 0.10256612689247868
8 - 0: 0.058092039068847336
8 - 1: -0.05809203906884733
9 - 0: -0.012301543705926545
9 - 1: 0.012301543705926538
10 - 0: -0.04798018170001507
10 - 1: 0.04798018170001506
11 - 0: 0.042013586085138895
11 - 1: -0.0420135860851389
12 - 0: 0.2741364763635855
12 - 1: -0.2741364763635855
13 - 0: -0.07928401412505649
13 - 1: 0.0792840141250565
14 - 0: -0.09954780131068686
14 - 1: 0.09954780131068684
15 - 0: 0.13559444310778
15 - 1: -0.13559444310778
16 - 0: 0.04074643090652586
16 - 1: -0.04074643090652586
17 - 0: -0.11481485312930884
17 - 1: 0.11481485312930884
